# Selects concrete columns with a certain amount of capacity ratio

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import comtypes.client

import json
import tkinter as tk
from tkinter import filedialog
import lib.csiapi as csiapi
import lib.util as util

%load_ext autoreload
%autoreload 2

In [3]:
# Initialization of ETABSObject
"""
ETABS_path_exe = r'C:/Program Files/Computers and Structures/ETABS 21_2/ETABS.exe'
if not os.path.isfile(ETABS_path_exe):
    root = tk.Tk()
    root.withdraw()
    ETABS_path = filedialog.askopenfilename(parent=root)
    root.destroy()
"""
# create API helper object
helper = comtypes.client.CreateObject("ETABSv1.Helper")
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
print("Succesfully created helper object")

try:
    #get the active ETABS object
    ETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    SapModel = ETABSObject.SapModel
    print("Succesfully created SapModel")
except (OSError, comtypes.COMError):  
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

# create folder for temporary csv files
csv_folder_path = os.getcwd() + r'\csv'
if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)

Succesfully created helper object
Succesfully created SapModel


In [49]:
SapModel.Analyze.RunAnalysis()
SapModel.DesignConcrete.StartDesign()

0

In [6]:
PMM_threshold = 1

concrete_column_table = csiapi.get_table_display(SapModel,'Concrete Column Design Summary - ACI 318-19')
os_concrete_column_table = concrete_column_table[concrete_column_table['PMMRatio'] > PMM_threshold]
os_concrete_column_uniquename = pd.Series(os_concrete_column_table['UniqueName']).unique()
for i in os_concrete_column_uniquename:
    SapModel.FrameObj.SetSelected(str(i.item()),True)

In [7]:
BCC_threshold = 1
JS_threshold = 1

concrete_joint_table = csiapi.get_table_display(SapModel,'Concrete Joint Design Summary - ACI 318-19')

os_bc = (concrete_joint_table['BCMajRatio'] > BCC_threshold) | (concrete_joint_table['BCMinRatio'] > BCC_threshold)
os_js = (concrete_joint_table['JSMajRatio'] > JS_threshold) | (concrete_joint_table['JSMinRatio'] > JS_threshold)
os_joint = os_bc | os_js

os_concrete_joint_table = concrete_joint_table[os_joint]

for i in pd.Series(os_concrete_joint_table['UniqueName']).unique():
    SapModel.FrameObj.SetSelected(str(i.item()),True)